In [1]:
import os
from dotenv import load_dotenv, find_dotenv
env_path = find_dotenv()
load_dotenv(env_path)
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CX = os.getenv('GOOGLE_CX')

#from google_images_search import GoogleImagesSearch
#from google.cloud import vision

In [2]:
def text_detection_full_response(path):
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    return response

In [74]:
def strip_menu(response):
    
    # remove these chars from entry
    chars_to_remove = '0123456789!"\'#$%&()*+,-./:;<=>?@[\]^_`{|}~♦●★‒…£¡™¢∞§¶•ªº–≠≠œ∑´®†¥¨≤≥÷ç√€'

    # remove entry if it exactly matches any of these
    drop_exact_words = ['sandwiches','restaurant','menu',
                        'restaurant menu','thank you','drinks',
                        'appetizer','appetizers','mains','dessert',
                        'side','sides','side order','breakfast','lunch'
                       'dinner','supper','starter','starters','local',
                        'fresh','food','main']
    
    # remove these words from entry
    words_to_remove = ['menu','restaurant','price','appetizer',
                       'appetizers','course','price','extra','extras']

    # remove entry if it contains any of these
    drop_contain_words = ['tax','consumer','advisory','illness']
    
    # remove entry if it starts with any of these
    drop_start_words = ['include','includes','including','lorem','with','and',
                       'served','serve']
    
    # drop entry if it contains fewer chars than minimum
    min_length = 4
    
    
    text = response.text_annotations[0].description
    menu_original = text.split('\n')
    
    menu_chars_removed = []
    for item in menu_original:
        for char in chars_to_remove:
            item = item.replace(char,'')
        menu_chars_removed.append(item)
        
    menu_exact_matches_dropped = []
    for item in menu_chars_removed:
        if item.lower() in drop_exact_words:
            pass
        else:
            menu_exact_matches_dropped.append(item)
        
    menu_words_removed = []
    for item in menu_exact_matches_dropped:
        temporary = []
        for word in item.split(' '):
            if word.lower() not in words_to_remove:
                temporary.append(word)
        remaining_words = ' '.join(temporary)
        menu_words_removed.append(remaining_words)
         
    menu_contains_dropped = []
    for item in menu_words_removed:
        temporary = []
        for word in item.split(' '):
            if word.lower() in drop_contain_words:
                temporary = []
                pass
            else:
                temporary.append(word)
        remaining_words = ' '.join(temporary)
        menu_contains_dropped.append(remaining_words)
        
    menu_starts_dropped = []
    for item in menu_contains_dropped:
        temporary = item.split(' ')
        if temporary[0].lower() in drop_start_words:
            pass
        else:
            menu_starts_dropped.append(item)
    
    menu_exact_matches_dropped = []
    for item in menu_starts_dropped:
        if item.lower() in drop_exact_words:
            pass
        else:
            menu_exact_matches_dropped.append(item)
            
    bounding_white_space_removed = [item.strip() for item in menu_exact_matches_dropped]
    too_short_dropped = [item for item in bounding_white_space_removed if len(item) >= min_length]
    stripped_menu = too_short_dropped
   
    print(menu_original)
    return(stripped_menu)

#

In [5]:
def optimized_image_fetch_and_check(query):
    from google_images_search import GoogleImagesSearch
    from google.cloud import vision
    
    print(f'searching for {query}...')
    print()

    gis = GoogleImagesSearch(GOOGLE_API_KEY,GOOGLE_CX)
    
    _search_params = {
    'q': f'{query} recipe',
    'num': 1,
    'imgSize': 'large',
    'imgType': 'photo',
    'imgColorType': 'color'}
    

    gis.search(search_params=_search_params)
    print('fetching image:')
    if len(gis.results()) == 0:
        print('no image found, not verified as food.')
        print()
        return None
    
    url = gis.results()[0].url
    print(url)
    print()
    
    verified_queries = ['cheeseburger','burger','pizza','fried chicken','ice cream sundae']
    
    if query.lower() in verified_queries:
        print(f'{query} already in known foods database, no need to verify!')
        print()
        return url
    
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = url
    
    response = client.label_detection(image=image, max_results=1)
    label = [lab.description for lab in response.label_annotations]
    score = [lab.score for lab in response.label_annotations]
    
    print('verification filter:')
    print('label must be Food')
    print('score must be above .96')
    print()
    print(f'label: {label}')
    print(f'label score: {score}')
    print()
    
    if label[0] == 'Food' and score[0] > .96:
        print('verified as food!')
        print()
        print(url)
        print()
        return url
    
    if label[0] in ['Food', 'Tableware']:
        text_response = client.text_detection(image=image)
        texts = text_response.text_annotations
        n_chars = 0
        if len(texts)>0:
            n_chars = len(texts[0].description)
    
    
        print('verification filter:')
        print('label must be Food or Tableware')
        print('score must be above .96')
        print('number of chars must be below 100')
        print()
    
        print(f'label: {label}')
        print(f'label score: {score}')
        print(f'chars detected: {n_chars}')
        print()
    
        if (label[0] == 'Food' or label[0] == 'Tableware') and score[0] > .95 and n_chars < 100:
            print('verified as food!')
            print()
            print(url)
            print()
            return url
    
    
    _search_params = {
    'q': f'{query} recipe',
    'num': 3,
    'imgSize': 'large',
    'imgType': 'photo',
    'imgColorType': 'color',
    'safe': 'medium'}
        
    
    gis = GoogleImagesSearch(GOOGLE_API_KEY,GOOGLE_CX)
    gis.search(search_params=_search_params)
    urls = [result.url for result in gis.results()]
    urls = urls[1:]
    print('fetching additional images:')
    for url in urls:
        print(url)
    print()
    
    print('verification filter:')
    print('label must be Food')
    print('score must be above .96')
    print()
    
    labels = []
    scores = []
    for url in urls:
        image.source.image_uri = url
        response = client.label_detection(image=image, max_results=1)
        label = [lab.description for lab in response.label_annotations]
        score = [lab.score for lab in response.label_annotations]
        labels.append(label)
        scores.append(score)
        
    print(f'labels: {labels}')
    print(f'label scores: {scores}')
    print()
    
    for label,score in zip(labels,scores):
        if label[0] == 'Food' and score[0] > .96:
            print('verified as food!')
            print()
            print(urls[labels.index(label)])
            print()
            return urls[labels.index(label)]
        
    if label[0] in ['Food', 'Tableware']:
        char_counts = []   
        for url in urls:
            text_response = client.text_detection(image=image)
            texts = text_response.text_annotations
            n_chars = 0
            if len(texts)>0:
                n_chars = len(texts[0].description)
            char_counts.append(n_chars)
        
        print('verification filter:')
        print('label must be Food or Tableware')
        print('score must be above .95')
        print('number of chars must be below 100')
        print()  
    
        print(f'labels: {labels}')
        print(f'label score: {scores}')
        print(f'chars detected: {char_counts}')
        print()
    
        for label,score,chars in zip(labels,scores,char_counts):
            if (label[0] == 'Food' or label[0] == 'Tableware') and score[0] > .95 and chars < 100:
                print('verified as food!')
                print()
                print(urls[labels.index(label)])
                print()
                return urls[labels.index(label)]
        
    print('not verified as food.')
    print()
    return None

#

In [ ]:
%%time
url = optimized_image_fetch_and_check('asodviaoivn')

In [7]:
path_start = '../raw_data/all_menus/'

paths = ['warung_sika_food.jpg',
'chinese_menu_1.png',
'english_menu_1.png',
'english_menu_2.jpg',
'english_menu_3.jpg',
'english_menu_4.jpg',
'english_menu_5.jpg',
'english_menu_6.jpg',
'english_menu_7.png',
'english_menu_8.png',
'english_menu_9.jpg',
'english_menu_10.jpg',
'english_menu_11.jpg',
'english_menu_12.png',
'indo_menu_1.png',
'saudi_menu_1.jpeg',
'spanish_menu_1.jpg',
'spanish_menu_2.jpg',
'thai_menu_1.png',
'warung_sika_drinks.jpg']

In [8]:
detected_menus = []
for path in paths:
    detected_menus.append(text_detection_full_response(path_start + path))

#

In [80]:
test_menu = detected_menus[19]
strip_menu(test_menu)

['HOT ICE TEA 5.', 'HOT/ICE JERUK 7', 'ICE LEMON TEA 7', 'BALI COFFEE 10', 'ICE COFFEE', 'FRESH JUICE 15-20', '15', 'HOT GINGER TEA 7', 'FRESH FRUITS 25', 'SOFTDRINKS', '5-10', 'MILKSHAKE KS 200', 'BINTANG', '25/35', 'BALT SIP25 45', ':']


['HOT ICE TEA',
 'HOTICE JERUK',
 'ICE LEMON TEA',
 'BALI COFFEE',
 'ICE COFFEE',
 'FRESH JUICE',
 'HOT GINGER TEA',
 'FRESH FRUITS',
 'SOFTDRINKS',
 'MILKSHAKE KS',
 'BINTANG',
 'BALT SIP']